# API SPOTIFY

In [1]:
pip install spotipy

   ---------------------------------------- 0.0/261.5 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/261.5 kB 660.6 kB/s eta 0:00:01
   --------------- ------------------------ 102.4/261.5 kB 1.2 MB/s eta 0:00:01
   ------------------------------------ --- 235.5/261.5 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 261.5/261.5 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials

client_id = "748f4cce341d436cb9ae583633bdafd6"
client_secret = "bdcc7c65a9244123a26d9472d3dc6865"
redirect_uri="http://localhost:8888/callback",
scope="playlist-read-private"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [34]:
import requests

CLIENT_ID = "748f4cce341d436cb9ae583633bdafd6"
CLIENT_SECRET = "bdcc7c65a9244123a26d9472d3dc6865"

url = "https://accounts.spotify.com/api/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET
}

response = requests.post(url, data=payload)
access_token = response.json().get("access_token")

print("Access Token:", access_token)


Access Token: BQA25Q-cGpQWz_ScR0HhyTj-OJSRm8ezw7MWzI96ghpWHpSuIbAg_v-ntPW0nGxY2pXq4BWNsiHMETzxBTNi1oELN51rWkuorLIiWJotRmLu8mlEgZonjJ3UDbX6Q5rui7xx60ktSMI


Docs API = https://spotipy.readthedocs.io/en/2.22.1/#spotipy.client.Spotify.category_playlists

In [5]:
import requests
import pandas as pd

# 👉 Reemplaza esto con tu Access Token válido
ACCESS_TOKEN1 = access_token

# Función para obtener canciones populares por año
def obtener_canciones_por_año(año):
    url = "https://api.spotify.com/v1/search"
    params = {
        "q": f"year:{año}",
        "type": "track",
        "limit": 50  # Puedes cambiar este valor para obtener más resultados
    }
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN1}"}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    canciones = []
    if "tracks" in data:
        for item in data["tracks"]["items"]:
            canciones.append({
                "nombre": item["name"],
                "artista": item["artists"][0]["name"],
                "id_artista": item["artists"][0]["id"],
                "año": año
            })
    return canciones

# Obtener datos para varios años
años = list(range(2010, 2026))  # Desde 2010 hasta 2025
canciones_por_año = []

for año in años:
    canciones_por_año.extend(obtener_canciones_por_año(año))

# Convertir a DataFrame
df_canciones = pd.DataFrame(canciones_por_año)
print(df_canciones.head())

# Guardar en CSV
df_canciones.to_csv("canciones_por_año.csv", index=False)


                         nombre       artista              id_artista   año
0          Just the Way You Are    Bruno Mars  0du5cEVh5yTK9QJze8zA0C  2010
1                    Angels Cry  Mariah Carey  4iHNK0tOyZPYnBU7nGAgpQ  2010
2      Only Girl (In The World)       Rihanna  5pKCCKE2ajJHZ9KAiaK11H  2010
3  The Real Slim Shady - Edited        Eminem  7dGJo4pcD2V6oG8kP0tJRR  2010
4                     The Spins    Mac Miller  4LLpKhyESsyAXpc4laK94U  2010


In [6]:
# Función para obtener el género del artista
def obtener_genero_artista(artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN1}"}

    response = requests.get(url, headers=headers)
    data = response.json()

    if "genres" in data and data["genres"]:
        return data["genres"][0]  # Tomamos el primer género
    return "Desconocido"

# Aplicar la función a todas las canciones
df_canciones["genero"] = df_canciones["id_artista"].apply(obtener_genero_artista)

# Guardar con géneros
df_canciones.to_csv("canciones_con_genero.csv", index=False)

print(df_canciones.head())


                         nombre       artista              id_artista   año  \
0          Just the Way You Are    Bruno Mars  0du5cEVh5yTK9QJze8zA0C  2010   
1                    Angels Cry  Mariah Carey  4iHNK0tOyZPYnBU7nGAgpQ  2010   
2      Only Girl (In The World)       Rihanna  5pKCCKE2ajJHZ9KAiaK11H  2010   
3  The Real Slim Shady - Edited        Eminem  7dGJo4pcD2V6oG8kP0tJRR  2010   
4                     The Spins    Mac Miller  4LLpKhyESsyAXpc4laK94U  2010   

        genero  
0  Desconocido  
1    christmas  
2  Desconocido  
3          rap  
4  Desconocido  


In [20]:
import pandas as pd
import requests
import base64

# Configuración de la API de Spotify
client_id = '748f4cce341d436cb9ae583633bdafd6'  # Reemplaza con tu Client ID de Spotify
client_secret = 'bdcc7c65a9244123a26d9472d3dc6865'  # Reemplaza con tu Client Secret de Spotify

# Función para obtener el token de acceso de Spotify
def obtener_token():
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {
        'Authorization': f'Basic {auth_header}'
    }
    data = {
        'grant_type': 'client_credentials'
    }
    response = requests.post(auth_url, headers=headers, data=data)
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception("Error al obtener el token de acceso")

# Función para buscar el género de un artista en Spotify
def buscar_genero_artista(artista, token):
    url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': f'Bearer {token}'
    }
    params = {
        'q': artista,
        'type': 'artist',
        'limit': 1
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        resultados = response.json()['artists']['items']
        if resultados:
            return resultados[0]['genres'][0] if resultados[0]['genres'] else None
    return None

# Leer el archivo CSV
df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\canciones_con_genero.csv")

# Obtener el token de acceso
token = obtener_token()

# Actualizar los géneros desconocidos
for index, row in df.iterrows():
    if row['genero'] == 'Desconocido':
        artista = row['artista']
        genero = buscar_genero_artista(artista, token)
        if genero:
            df.at[index, 'genero'] = genero
            print(f"Género actualizado para {artista}: {genero}")
        else:
            print(f"No se encontró género para {artista}")

# Guardar el archivo actualizado
df.to_csv('canciones_con_genero_actualizado.csv', index=False)

print("Archivo actualizado guardado como 'canciones_con_genero_actualizado.csv'")

No se encontró género para Bruno Mars
No se encontró género para Rihanna
No se encontró género para Mac Miller
No se encontró género para Michael Jackson
No se encontró género para Rihanna
No se encontró género para Avril Lavigne
No se encontró género para Bruno Mars
No se encontró género para Lily Allen
No se encontró género para Bruno Mars
No se encontró género para Christina Aguilera
No se encontró género para Kesha
No se encontró género para Voltaj
No se encontró género para Patrick Watson
No se encontró género para Michael Jackson
No se encontró género para Mike Posner
No se encontró género para Bruno Mars
No se encontró género para Michael Jackson
No se encontró género para Flo Rida
No se encontró género para Michael Jackson
No se encontró género para P!nk
No se encontró género para Edita
No se encontró género para Black Eyed Peas
No se encontró género para Gorillaz
No se encontró género para Pitbull
No se encontró género para JayDee
No se encontró género para Kazik
No se encontr

In [22]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\canciones_con_genero.csv")

# Filtrar los artistas con género "Desconocido"
artistas_desconocidos = df[df['genero'] == 'Desconocido']['artista'].unique()

# Mostrar los artistas con género desconocido
print("Artistas con género desconocido:")
for artista in artistas_desconocidos:
    print(artista)

# Guardar la lista en un archivo CSV (opcional)
artistas_desconocidos_df = pd.DataFrame(artistas_desconocidos, columns=['artista'])
artistas_desconocidos_df.to_csv('artistas_desconocidos.csv', index=False)

print("\nLista de artistas con género desconocido guardada en 'artistas_desconocidos.csv'")

Artistas con género desconocido:
Bruno Mars
Rihanna
Mac Miller
Michael Jackson
Avril Lavigne
Lily Allen
Christina Aguilera
Kesha
Voltaj
Patrick Watson
Mike Posner
Flo Rida
P!nk
Edita
Black Eyed Peas
Gorillaz
Pitbull
JayDee
Kazik
Gotye
Foster The People
Coldplay
Beyoncé
Revolver
Childish Gambino
Stereosoul
Frank Ocean
M83
Cold Chisel
Basta
Jessie J
Jennifer Lopez
Mundy
Macklemore
Abhishek - Akshay
Ne-Yo
Specktors
Lana Del Rey
The Script
Carly Rae Jepsen
Imagine Dragons
秦 基博 meets 坂道のアポロン
MKTO
Kali
Nicki Minaj
Icona Pop
One Direction
Tom Odell
Pezet
Abel The Kid
Wale
The Neighbourhood
Milky Chance
Miley Cyrus
Santigold
Bastille
Lorde
Naughty Boy
Kodaline
Hozier
WALK THE MOON
George Ezra
Patrick Sébastien
TV Girl
Dove Cameron
Lipo
Mr. Probz
Kid Ink
Diggy
Taylor Swift
Lil Dicky
Lord Huron
sana
White Noise Baby Sleep
Geegun
James Bay
Fem Fel
Mark Ronson
OMI
Van Morrison
Justin Bieber
Fetty Wap
Ain't No Love
The Weeknd
Olly Alexander (Years & Years)
Joe Satriani
Jula
Hayley Kiyoko
Volo
The C

In [ ]:
df_Generos_artistas_desconocido = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\DATA\\artistas_desconocidos.csv")
df_Generos_artistas_desconocido 

,artista
0,Bruno Mars
1,Rihanna
2,Mac Miller
3,Michael Jackson
4,Avril Lavigne
...,...
230,Samara Cyn
231,Maya Delilah
232,Leven Kali
233,Elohim


In [41]:
import pandas as pd

# Supongamos que tienes un DataFrame con los artistas desconocidos
df_Generos_artistas_desconocido = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\artistas_desconocidos.csv")

# Diccionario con los géneros correctos (puedes crearlo manualmente o cargarlo desde un archivo)
generos_correctos = {
    "Bruno Mars": "pop","Rihanna": "pop","Michael Jackson": "pop","Lily Allen": "pop","Patrick Watson": "indie", "Mac Miller" :"hip-hop","Avril Lavigne" : "pop", "Christina Aguilera":"pop","Kesha" : "pop","Voltaj":"rock","Mike Posner":"pop","Flo Rida":"hip-hop","P!nk":"pop","Edita":"pop","Black Eyed Peas":"hip-hop","Gorillaz":"rock","Pitbull":"latino","JayDee":"hip-hop","Kazik":"rock","Gotye":"indie",
    "Coldplay": "alternative rock",
    "Beyoncé": "R&B",
    "Revolver": "rock en español",
    "Childish Gambino": "hip-hop",
    "Stereosoul": "electronic",
    "Frank Ocean": "R&B",
    "M83": "electronic",
    "Cold Chisel": "rock",
    "Basta": "hip-hop",
    "Jessie J": "pop",
    "Jennifer Lopez": "pop",
    "Mundy": "folk",
    "Macklemore": "hip-hop",
    "Abhishek - Akshay": "bollywood",
    "Ne-Yo": "R&B",
    "Specktors": "hip-hop",
    "Lana Del Rey": "indie pop",
    "The Script": "pop rock",
    "Carly Rae Jepsen": "pop",
    "Imagine Dragons": "alternative rock",
    "秦 基博 meets 坂道のアポロン": "j-pop",
    "MKTO": "pop",
    "Kali": "hip-hop",
    "Nicki Minaj": "hip-hop",
    "Icona Pop": "electropop",
    "One Direction": "pop",
    "Tom Odell": "indie pop",
    "Pezet": "hip-hop",
    "Abel The Kid": "hip-hop",
    "Wale": "hip-hop",
    "The Neighbourhood": "indie rock",
    "Milky Chance": "indie pop",
    "Miley Cyrus": "pop",
    "Santigold": "alternative",
    "Bastille": "indie pop",
    "Lorde": "indie pop",
    "Naughty Boy": "pop",
    "Kodaline": "indie pop",
    "Hozier": "indie folk",
    "WALK THE MOON": "indie pop",
    "George Ezra": "indie folk",
    "Patrick Sébastien": "variété française",
    "TV Girl": "indie pop",
    "Dove Cameron": "pop",
    "Lipo": "electronic",
    "Mr. Probz": "pop",
    "Kid Ink": "hip-hop",
    "Diggy": "hip-hop",
    "Taylor Swift": "pop",
    "Lil Dicky": "hip-hop",
    "Lord Huron": "indie folk",
    "sana": "j-pop",
    "White Noise Baby Sleep": "ambient",
    "Geegun": "hip-hop",
    "James Bay": "indie pop",
    "Fem Fel": "hip-hop",
    "Mark Ronson": "pop",
    "OMI": "pop",
    "Van Morrison": "rock",
    "Justin Bieber": "pop",
    "Fetty Wap": "hip-hop",
    "Ain't No Love": "R&B",
    "The Weeknd": "R&B",
    "Olly Alexander (Years & Years)": "electropop",
    "Joe Satriani": "rock",
    "Jula": "pop",
    "Hayley Kiyoko": "pop",
    "Volo": "hip-hop",
    "The Chainsmokers": "edm",
    "MGK": "hip-hop",
    "Sia": "pop",
    "Oh Wonder": "indie pop",
    "Dwayne Johnson": "pop",
    "DAWN": "pop",
    "Adam French": "indie pop",
    "Roxette": "pop rock",
    "DJ Zet": "electronic",
    "DJ Snake": "edm",
    "Tyler, The Creator": "hip-hop",
    "Declan McKenna": "indie pop",
    "Beatsteaks": "rock",
    "Sonny Smith": "indie",
    "Richy Mitch & The Coal Miners": "indie",
    "Zara Larsson": "pop",
    "Niall Horan": "pop",
    "衛蘭": "cantopop",
    "French Montana": "hip-hop",
    "Laineen Kasperi": "hip-hop",
    "Chase Atlantic": "indie pop",
    "Revolution": "rock",
    "Picture This": "indie pop",
    "Halsey": "pop",
    "Gerry Cinnamon": "indie",
    "Carlos Marco": "pop",
    "Steve Lacy": "R&B",
    "Sabrina Carpenter": "pop",
    "Harry Styles": "pop",
    "Alexander Stewart": "pop",
    "KNEECAP": "hip-hop",
    "Simply Red": "pop",
    "Billie Eilish": "indie pop",
    "Austin Brown": "pop",
    "Pretty Pink": "edm",
    "Dominic Fike": "indie pop",
    "Little Big": "electronic",
    "benny blanco": "pop",
    "Begi Lotfi": "pop",
    "Lauv": "pop",
    "Sleiman": "hip-hop",
    "Shannon & the Clams": "indie rock",
    "5 Seconds of Summer": "pop rock",
    "fhop music": "gospel",
    "Jonny Greenwood": "experimental",
    "Grace Hodgett Young": "indie",
    "K27": "hip-hop",
    "Louise": "pop",
    "Janie Bay": "pop",
    "Frans": "pop",
    "SYU(from GALNERYUS)": "metal",
    "Post Malone": "hip-hop",
    "Lil Yase": "hip-hop",
    "Sam Fender": "indie rock",
    "Chelsea Cutler": "indie pop",
    "Elias Abbas": "hip-hop",
    "Jeff Buckley": "rock",
    "Andy Grammer": "pop",
    "Alvaro Diaz": "hip-hop",
    "Dean Lewis": "indie pop",
    "Chi City": "hip-hop",
    "Noah Kahan": "indie folk",
    "Julia Michaels": "pop",
    "Dream Supplier": "ambient",
    "Gracie Abrams": "indie pop",
    "Blackie and the Rodeo Kings": "rock",
    "Cherrie": "hip-hop",
    "Karen Zoid": "rock",
    "Glass Animals": "indie pop",
    "Conan Gray": "indie pop",
    "Noah Cyrus": "pop",
    "Ricky Rich": "hip-hop",
    "Tate McRae": "pop",
    "Fran": "pop",
    "Adrianne Lenker": "indie folk",
    "Nuno Ribeiro": "pop",
    "Hotel Ugly": "indie pop",
    "SAMIRA": "pop",
    "Ava Max": "pop",
    "Kukon": "hip-hop",
    "Lola Young": "indie pop",
    "Nek": "pop",
    "Olivia Rodrigo": "pop",
    "Greg Yeya": "hip-hop",
    "ame": "electronic",
    "Lou": "pop",
    "Tory Lanez": "hip-hop",
    "Rouss": "pop",
    "The Noisy Outdoors": "ambient",
    "Belters Only": "edm",
    "Relaxed Minds": "ambient",
    "Baby Keem": "hip-hop",
    "Jay Oc": "hip-hop",
    "Karima": "pop",
    "Alexy Large": "hip-hop",
    "D.T.E": "hip-hop",
    "Milli Hughes": "ambient",
    "††† (Crosses)": "experimental",
    "Dima Bilan": "pop",
    "CQ Wrestling": "rock",
    "YoungBoy Never Broke Again": "hip-hop",
    "Lu Decker": "pop",
    "Rod D": "hip-hop",
    "Lizzy McAlpine": "indie pop",
    "Folder": "electronic",
    "LF SYSTEM": "edm",
    "La Peee": "hip-hop",
    "Johannes Johansson": "ambient",
    "Lah Pat": "hip-hop",
    "Chappell Roan": "pop",
    "Jamie Brown": "pop",
    "Teddy Swims": "pop",
    "Slim": "hip-hop",
    "cassö": "edm",
    "Einar": "hip-hop",
    "The Kid LAROI": "hip-hop",
    "El Mimoso Luis Antonio López": "regional mexican",
    "Amble": "indie",
    "CYRIL": "pop",
    "The Last Dinner Party": "indie",
    "LEMONY": "indie",
    "馬念先": "mandopop",
    "Gigi Perez": "indie",
    "Aminé": "hip-hop",
    "SAINt JHN": "hip-hop",
    "Money Man": "hip-hop",
    "FINNEAS": "indie pop",
    "Chrystal": "pop",
    "Doechii": "hip-hop",
    "Tay Savage": "hip-hop",
    "Mackenzy Mackay": "indie",
    "Benson Boone": "pop",
    "Myles Smith": "indie",
    "Alex Warren": "indie",
    "BL3SS": "pop",
    "SIENNA SPIRO": "indie",
    "Calum Bowie": "indie",
    "Kabin Crew": "hip-hop",
    "Jessie Reyez": "indie pop",
    "Jeannie C. Riley": "country",
    "Tamino": "indie",
    "Samara Cyn": "indie",
    "Maya Delilah": "indie",
    "Leven Kali": "R&B",
    "Elohim": "electronic",
    "Marti Perramon": "pop",
    "Foster The People":"indie"
}

# Añadir la columna "genero" al DataFrame
df_Generos_artistas_desconocido['genero'] = df_Generos_artistas_desconocido['artista'].map(generos_correctos)

# Mostrar el DataFrame actualizado
print(df_Generos_artistas_desconocido)

# Guardar el DataFrame actualizado en un nuevo archivo CSV (opcional)
df_Generos_artistas_desconocido.to_csv('artistas_desconocidos_con_genero.csv', index=False)

             artista      genero
0         Bruno Mars         pop
1            Rihanna         pop
2         Mac Miller     hip-hop
3    Michael Jackson         pop
4      Avril Lavigne         pop
..               ...         ...
230       Samara Cyn       indie
231     Maya Delilah       indie
232       Leven Kali         R&B
233           Elohim  electronic
234   Marti Perramon         pop

[235 rows x 2 columns]


In [42]:
df_generos1 = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\artistas_desconocidos_con_genero.csv")
df_generos1

,artista,genero
0,Bruno Mars,pop
1,Rihanna,pop
2,Mac Miller,hip-hop
3,Michael Jackson,pop
4,Avril Lavigne,pop
...,...,...
230,Samara Cyn,indie
231,Maya Delilah,indie
232,Leven Kali,R&B
233,Elohim,electronic


In [43]:
import pandas as pd

# Leer el archivo original
df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\canciones_con_genero.csv")

# Leer el archivo con los géneros correctos
generos_correctos_df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\artistas_desconocidos_con_genero.csv")

# Crear un diccionario con los géneros correctos
generos_dict = dict(zip(generos_correctos_df['artista'], generos_correctos_df['genero']))

# Función para actualizar el género
def actualizar_genero(row):
    if row['genero'] == 'Desconocido' and row['artista'] in generos_dict:
        return generos_dict[row['artista']]
    return row['genero']

# Aplicar la función a la columna "genero"
df['genero'] = df.apply(actualizar_genero, axis=1)

# Guardar el archivo actualizado
df.to_csv('Datamatrix.csv', index=False)

print("Archivo actualizado guardado como 'Datamatrix.csv'")

Archivo actualizado guardado como 'Datamatrix.csv'


In [44]:
df_datamatrix = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\Datamatrix.csv")
df_datamatrix

,nombre,artista,id_artista,año,genero
0,Just the Way You Are,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,2010,pop
1,Angels Cry,Mariah Carey,4iHNK0tOyZPYnBU7nGAgpQ,2010,christmas
2,Only Girl (In The World),Rihanna,5pKCCKE2ajJHZ9KAiaK11H,2010,pop
3,The Real Slim Shady - Edited,Eminem,7dGJo4pcD2V6oG8kP0tJRR,2010,rap
4,The Spins,Mac Miller,4LLpKhyESsyAXpc4laK94U,2010,hip-hop
...,...,...,...,...,...
793,Are U Still - Live,Leven Kali,5YZ5AExR68U3ZblH6HcO6B,2025,R&B
794,Greenlight,Tate McRae,45dkTj5sMRSjrmBSBeiHym,2025,pop
795,Scratch My Brain (Interlude) (Visualizer),Elohim,6wKxOKEA3K6R2UZ3COLXEY,2025,electronic
796,Means I care,Tate McRae,45dkTj5sMRSjrmBSBeiHym,2025,pop


In [45]:
# Contar la frecuencia de cada género por año
df_frecuencia = df_canciones.groupby(["año", "genero"]).size().reset_index(name="frecuencia")

# Guardar los datos
df_frecuencia.to_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\Datamatrix.csv", index=False)

# Mostrar los 10 géneros más frecuentes por año
print(df_frecuencia.sort_values(["año", "frecuencia"], ascending=[True, False]).groupby("año").head(10))


      año         genero  frecuencia
0    2010    Desconocido          22
17   2010            rap           6
15   2010            pop           3
1    2010      christmas           2
4    2010            edm           2
..    ...            ...         ...
283  2025        country           1
284  2025   gujarati pop           1
285  2025     indie folk           1
286  2025        mizrahi           1
287  2025  northern soul           1

[160 rows x 3 columns]


In [54]:
# Diccionario para agrupar géneros en categorías principales
mapa_generos = {
    "pop": ["pop", "dance pop", "christmas pop", "electropop","christmas","soft pop","indonesian pop","nederpop","art pop","schlager","baroque pop","cantopop","mandopop","britpop","german pop","french pop","schlagerparty","chamber pop","brazilian pop","norwegian pop","bedroom pop","iskelmä","psychedelic pop","city pop","pop urbaine","gujarati pop"],
    "rock": ["rock","pop-rock","pop rock", "alternative rock", "hard rock","folk rock","psychedelic rock","rock en español","post-rock","grunge","modern rock","pop punk","emo","classic rock","neue deutsche welle","pinoy alternative","new wave","brazilian rock","thai rock","post-punk","shoegaze"],
    "hip-hop": ["hip hop","hip-hop","rap", "trap", "gangsta rap","experimental hip hop","german hip hop","east coast hip hop","brazilian hip hop","melodic rap","emo rap","italian trap","rage rap","egyptian hip hop","grime","argentine trap","trap latino","boom bap","uk drill","brooklyn drill"],
    "electrónica": ["edm", "house", "techno", "trance", "drum and bass","jungle","minimal techno","tropical house","future bass","electronica","tech house","cedm","idm","french house","electroclash","electronic","electropop"],
    "latino": ["reggaeton", "latin pop", "salsa", "bachata", "tropical","latin alternative","regional mexican","bolero","cumbia","champeta","reggaeton chileno","rkt","latin","latino"],
    "metal": ["metal", "death metal", "black metal", "metalcore","symphonic metal","nu metal","melodic hardcore","slowcore","horrorcore","nightcore"],
    "jazz": ["jazz", "smooth jazz", "cool jazz","french jazz","doo-wop"],
    "clásica": ["classical", "baroque", "orchestral","opera","neoclassical","choral","chamber music"],
    "folk": ["folk", "indie folk", "americana", "folk pop","phleng phuea chiwit","christian folk"],
    "blues": ["blues", "delta blues", "chicago blues","southern gospel","retro soul","soul","gospel","northern soul"],
    "country": ["country", "modern country", "classic country","bluegrass"],
    "reggae": ["reggae", "dancehall", "roots reggae","spanish-language reggae"],
    "r&b": ["r&b","alternative r&b","R&B"],
    "samba" : ["pagode", "samba", "axé","mpb","pagode baiano","sertanejo","sertanejo tradicional","forró","nova mpb","agronejo"],
    "africano" : ["afrobeat", "afropop","kizomba","afrobeats"],
    "k-pop" : ["k-pop", "k-hip hop","thai pop","c-pop","t-pop","mandopop","j-pop","anime"],
    "indie" : ["indie pop", "indie folk", "indie rock","indie","piano house","tamil indie","indie","alternative","experimental"],
    "bollywood" : ["bollywood","bangla pop","mollywood"],
    "dance" : ["post-disco","italo disco"],
    "flamenco" : ["flamenco","flamenco pop"],
    "ambient" : ["ambient"]

}


In [55]:
# Función para asignar género general
def asignar_genero_general(genero_especifico):
    for categoria, lista_generos in mapa_generos.items():
        if genero_especifico in lista_generos:
            return categoria
    return "Otros"  # Si no encontramos coincidencia, lo ponemos en "Otros"

# Aplicar la función a la columna de géneros
df_datamatrix["genero_general"] = df_datamatrix["genero"].apply(asignar_genero_general)

# Guardar en CSV
df_datamatrix.to_csv("canciones_con_genero_general.csv", index=False)
print(df_datamatrix[["nombre","artista","año","genero", "genero_general"]].head(20))


                                       nombre          artista   año  \
0                        Just the Way You Are       Bruno Mars  2010   
1                                  Angels Cry     Mariah Carey  2010   
2                    Only Girl (In The World)          Rihanna  2010   
3                The Real Slim Shady - Edited           Eminem  2010   
4                                   The Spins       Mac Miller  2010   
5                             Smooth Criminal  Michael Jackson  2010   
6                                         S&M          Rihanna  2010   
7              Estuve a punto de - Entrevista      Miguel Bosé  2010   
8   DJ Got Us Fallin' In Love (feat. Pitbull)            USHER  2010   
9                                       Alice    Avril Lavigne  2010   
10                                  Marry You       Bruno Mars  2010   
11                                 Let You Go   Chase & Status  2010   
12                                    Runaway       Kanye West  

In [ ]:
# Contar la frecuencia de cada género general por año
df_frecuencia = df_datamatrix.groupby(["año", "genero_general"]).size().reset_index(name="frecuencia")

# Guardar los datos
df_frecuencia.to_csv("generos_mas_populares_por_año.csv", index=False)

# Mostrar los 5 géneros más populares por año
print(df_frecuencia.sort_values(["año", "frecuencia"], ascending=[True, False]).groupby("año").head(5))




      año genero_general  frecuencia
5    2010            pop          25
1    2010        hip-hop           9
0    2010    electrónica           3
7    2010           rock           3
8    2010          samba           3
..    ...            ...         ...
200  2025            pop          22
196  2025          indie           9
195  2025        hip-hop           4
192  2025        country           3
189  2025          Otros           2

[80 rows x 3 columns]


In [58]:
df_frecuencia.to_csv("Datamatrix_genres.csv")

In [57]:
# Revisar los géneros que quedaron en "Otros"
generos_otros = df_datamatrix[df_datamatrix["genero_general"] == "Otros"]["genero"].unique()
print(generos_otros)


['lullaby' 'oyun havasi' 'celtic' 'singer-songwriter' 'comedy'
 "children's music" 'christian' 'variété française' 'avant-garde'
 'medieval' 'devotional' 'sholawat' 'italian singer-songwriter' 'opm']
